In [ ]:
# Load the data
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr
from scipy.stats import norm, ks_2samp  # no scipy - comment out

%matplotlib inline

# Introduction

In this notebook, we will walk through a hacker's approach to statistical thinking, as applied to network analysis.

## Statistics in a Nutshell

All of statistics can be broken down into two activities:

- Descriptively summarizing data. (a.k.a. **"descriptive statistics"**)
- Figuring out whether something happened by random chance. (a.k.a. **"inferential statistics"**)

### Descriptive Statistics

- Centrality measures: mean, median, mode
- Variance measures: inter-quartile range (IQR), variance and standard deviation

### Inferential Statistics

- Models of Randomness (see below)
- Hypothesis Testing
- Fitting Statistical Models

**Models of Randomness**

*Discrete Distributions:*

- Bernoulli: probability p of success in 1 try (e.g. one flip of coin).
- Binomial: probability p of success given n number of tries. `n * bernoulli trials` follows binomial distribution.
- Poisson: processes with a per-unit rate.

*Continuous Distributions:*

- Uniform: equal probability over the range of probable values. Can also be made discrete.
- Normal: everyone's favourite.

## Playing with Real Data!

In this notebook, we will statistically test whether a protein-protein interaction network follows an Erdos-Renyi random graph model or not. We will also see how to load real data into a NetworkX graph.

In [ ]:
# Take a quick peek at what the edge list looks like.
!head datasets/moreno_propro/out.moreno_propro_propro.txt

In [ ]:
# Read in the data.
# Note from above that we have to skip the first two rows, and that there's no header column,and that the edges are
# delimited by spaces in between the nodes. Hence the syntax below:
propro = pd.read_csv('datasets/moreno_propro/out.moreno_propro_propro.txt', skiprows=2, header=None, delimiter=' ')
propro.columns = ['prot1_id', 'prot2_id']
propro.head()
len(propro)

**Exercise:** Put to work your knowledge of how the `networkx` API works. Add these edges into the graph.

In [ ]:
proproG = nx.Gra ph()
proproG.add_edges_from(zip(propro['prot1_id'], propro['prot2_id']))
len(proproG.edges()) 

**Exercise:** Compute some basic descriptive statistics about the graph, namely:

- the number of nodes,
- the number of edges,
- the graph density,
- the distribution of degree centralities in the graph,


In [ ]:
# Number of nodes:
len(proproG.nodes())

In [ ]:
# Number of edges:
len(proproG.edges())

In [ ]:
# Graph density:
# 
nx.density(proproG)

In [ ]:
# Degree centrality distribution:
list(nx.degree_centrality(proproG).values())

**Exercise:** Make a histogram of the degree centralities for the protein-protein interaction graph, and the E-R graph.

In [ ]:
ppG_deg_centralities = list(nx.degree_centrality(proproG).values())
n, bins, patches = plt.hist(ppG_deg_centralities)

In [ ]:
erG = nx.erdos_renyi_graph(n=len(proproG.nodes()), p=nx.density(proproG))
erG_deg_centralities = list(nx.degree_centrality(erG).values())
n, bins, patches = plt.hist(erG_deg_centralities)

From visualizing these two distributions, it is clear that they look very different. How do we quantify this difference, and statistically test whether the protein-protein graph could have arisen under an Erdos-Renyi model?

**Discuss this with your neighbors.**

One thing we might observe is that the variance, that is the "spread" around the mean, differs between the E-R model compared to our data. Therefore, we can compare variance of the data to the distribtion of variances under an E-R model.

This is essentially following the logic of statistical inference by 'hacking' (not to be confused with the statistical bad practice of p-hacking).

**Exercise:** Fill in the skeleton code below to simulate 100 E-R graphs.

In [ ]:
# 1. Generate 100 E-R graph degree centrality variance measurements and store them.
# Takes ~50 seconds or so.
er_vars = np.zeros(100)  # variances for 1000 simulaed E-R graphs.
for i in range(100):
    erG = nx.erdos_renyi_graph(n=len(proproG.nodes()), p=nx.density(proproG))
    erG_deg_centralities = list(nx.degree_centrality(erG).values())
    er_vars[i] = np.var(erG_deg_centralities)

In [ ]:
# 2. Compute the test statistic that is going to be used for the hypothesis test.
# Hint: numpy has a "var" function implemented that computes the variance of a distribution of data.
ppG_var = np.var(ppG_deg_centralities)

In [ ]:
# Do a quick visual check
n, bins, patches = plt.hist(er_vars)
plt.vlines(ppG_var, ymin=0, ymax=max(n), color='red', lw=2)

Visually, it should be quite evident that the protein-protein graph did not come from an E-R distribution. Statistically, we can also use the hypothesis test procedure to quantitatively test this, using our simulated E-R data, 

In [ ]:
# Conduct the hypothesis test.
ppG_var > np.percentile(er_vars, 99)  # we can only use the 99th percentile, because there are only 100 data points.

Another way to do this is to use the 2-sample Kolmogorov-Smirnov test implemented in the `scipy.stats` module. From the docs:

> This tests whether 2 samples are drawn from the same distribution. Note
that, like in the case of the one-sample K-S test, the distribution is
assumed to be continuous.
>
> This is the two-sided test, one-sided tests are not implemented.
The test uses the two-sided asymptotic Kolmogorov-Smirnov distribution.
>
> If the K-S statistic is small or the p-value is high, then we cannot
reject the hypothesis that the distributions of the two samples
are the same.

As an example to convince yourself that this test works, run the synthetic examples below.

In [ ]:
# Scenario 1: Data come from the same distributions.
# Notice the size of the p-value.
dist1 = npr.random(size=(100))
dist2 = npr.random(size=(100))

ks_2samp(dist1, dist2)
# dist1

In [ ]:
# Scenario 2: Data come from different distributions. 
# Note the size of the KS statistic, and the p-value.

dist1 = norm(3, 1).rvs(100)
dist2 = norm(5, 1).rvs(100)

ks_2samp(dist1, dist2)

**Exercise:** Now, conduct the K-S test for one synthetic graph and the data.

In [ ]:
# Now try it on the data distribution
ks_2samp(erG_deg_centralities, ppG_deg_centralities)

Networks may be high-dimensional objects, but the logic for inference on network data essentially follows the same logic as for 'regular' data:

- Identify a model of 'randomness' that may model how your data may have been generated.
- Compute a "test statistic" for your data and the model.
- Compute the probability of observing the data's test statistic under the model.

# Further Reading

Jake Vanderplas' "Statistics for Hackers" slides: https://speakerdeck.com/jakevdp/statistics-for-hackers

Allen Downey's "There is Only One Test": http://allendowney.blogspot.com/2011/05/there-is-only-one-test.html